In [1]:
import BMTextTokenizer as bmt
import pandas as pd
import networkx as nx
import time
import matplotlib.pyplot as plt
import stanza
from itertools import permutations
%matplotlib inline

2022-02-05 11:49:31 INFO: Loading these models for language: en (English):
| Processor | Package    |
--------------------------
| tokenize  | craft      |
| pos       | craft      |
| lemma     | craft      |
| depparse  | craft      |
| ner       | bionlp13cg |

2022-02-05 11:49:31 INFO: Use device: cpu
2022-02-05 11:49:31 INFO: Loading: tokenize
2022-02-05 11:49:31 INFO: Loading: pos
2022-02-05 11:49:31 INFO: Loading: lemma
2022-02-05 11:49:31 INFO: Loading: depparse
2022-02-05 11:49:31 INFO: Loading: ner
2022-02-05 11:49:31 INFO: Done loading processors!
2022-02-05 11:49:31 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |

2022-02-05 11:49:31 INFO: Use device: cpu
2022-02-05 11:49:31 INFO: Loading: tokenize
2022-02-05 11:49:31 INFO: Loading: pos
2022-02-05 11:49:31 INFO: Loading: lemma
2022-02-05 11:49:31 INFO: Loading: depparse
2022-

In [2]:
# Default stanza pipeline to be used throughout this notebook
nlp = stanza.Pipeline('en', package='craft', processors={'ner': 'BioNLP13CG'})
pretokenized_nlp = stanza.Pipeline(lang='en', package='craft', processor='tokenize', tokenize_pretokenized=True) 

2022-02-05 11:49:34 INFO: Loading these models for language: en (English):
| Processor | Package    |
--------------------------
| tokenize  | craft      |
| pos       | craft      |
| lemma     | craft      |
| depparse  | craft      |
| ner       | bionlp13cg |

2022-02-05 11:49:34 INFO: Use device: cpu
2022-02-05 11:49:34 INFO: Loading: tokenize
2022-02-05 11:49:34 INFO: Loading: pos
2022-02-05 11:49:34 INFO: Loading: lemma
2022-02-05 11:49:34 INFO: Loading: depparse
2022-02-05 11:49:34 INFO: Loading: ner
2022-02-05 11:49:35 INFO: Done loading processors!
2022-02-05 11:49:35 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |

2022-02-05 11:49:35 INFO: Use device: cpu
2022-02-05 11:49:35 INFO: Loading: tokenize
2022-02-05 11:49:35 INFO: Loading: pos
2022-02-05 11:49:35 INFO: Loading: lemma
2022-02-05 11:49:35 INFO: Loading: depparse
2022-

In [3]:
path = '/Volumes/GoogleDrive/My Drive/_My Data Analytics Exercise/Exercise/Biopython/CE/'

In [4]:
%%time
df = pd.read_csv(path+'all_interactions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124 entries, 0 to 1123
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   interactionId  1124 non-null   object
 1   sentence       1124 non-null   object
 2   entities       1124 non-null   object
 3   interaction    1124 non-null   object
 4   relationship   1124 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 44.0+ KB
CPU times: user 18.5 ms, sys: 4.79 ms, total: 23.3 ms
Wall time: 29.8 ms


In [5]:
%%time
pos_tag = []
with open('upos_tag.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        pos_tag.append(line.replace('\n',''))
pos_tag       

CPU times: user 1.53 ms, sys: 3.41 ms, total: 4.94 ms
Wall time: 13.8 ms


['VERB',
 'NOUN',
 'ADJ',
 'NUM',
 'ADP',
 'PUNCT',
 'CONJ',
 'DET',
 'PROPN',
 'PRON',
 'SCONJ',
 'SYM',
 'ADV',
 'AUX',
 'PART',
 'X',
 'CCONJ']

In [6]:
with_interactions = df[df['relationship']==1].copy()
without_interactions = df[df['relationship']==0].copy()

In [8]:
with_interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825 entries, 1 to 1122
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   interactionId  825 non-null    object
 1   sentence       825 non-null    object
 2   entities       825 non-null    object
 3   interaction    825 non-null    object
 4   relationship   825 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 38.7+ KB


In [9]:
without_interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 1123
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   interactionId  299 non-null    object
 1   sentence       299 non-null    object
 2   entities       299 non-null    object
 3   interaction    299 non-null    object
 4   relationship   299 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 14.0+ KB


In [10]:
without_interactions.head()

,interactionId,sentence,entities,interaction,relationship
0,BioInfer_d125,Calculated solution molecular masses are 213 k...,"[('alpha-catenin#', 'T1', 'Gene/protein/RNA', ...",{},0
2,BioInfer_d544,Smooth muscle alpha actin and myosin heavy cha...,"[('Smooth muscle alpha actin#', 'T1', 'Gene/pr...",{},0
4,BioInfer_d232,Genetic dissection of Drosophila myofibril for...,"[('Act88F#', 'T6', 'Individual_protein', (146,...",{},0
5,BioInfer_d653,"The nucleotide sequences of nucleocapsid (N), ...","[('F#', 'T1', 'Gene/protein/RNA', (287, 288)),...",{},0
6,BioInfer_d28,"After 24 or 48 hours of shear, staining for VE...","[('alpha-catenin#', 'T6', 'Gene/protein/RNA', ...",{},0


### Preprocessing of Sentences for further analysis

In [13]:
%%time
# Those sentences that have NO relationship indicated in original data set

directory = '/Volumes/GoogleDrive/My Drive/_My Data Analytics Exercise/Exercise/Biopython/CE/without/'
from tqdm import tqdm
from random import sample
tokens_lst = dict()
index = sample(sorted(without_interactions.index),100)
special_attention = []
for i in tqdm(index):
    ids = without_interactions.loc[i, 'interactionId']
    sent = bmt.cleanSentencefmDF(without_interactions.loc[i, 'sentence'])
    print (i, sent)
    ents = without_interactions.loc[i, 'entities']
    doc = bmt.dynamicTokenizer(pretokenized_nlp, sent) # retokenize sentence to group chunks into single meaningful biomedical entity
    entities = bmt.extractEntities(bmt.extractEntitiesfmDF(ents))
    #print ('Number of unique entity:', len(entities), entities)
    H = bmt.createTokenGraphwithContraction(doc) # create an undirected graph connecting various tokens
    filename = directory+ids+".gpickle"
    nx.write_gpickle(H, filename)
    tokens = bmt.extractTokensOfInterestfmGraph(H, entities, threshold=0.8)
    #print ('Number of Tokens:', len(tokens), tokens)
    tokens_lst[ids] = tokens

  0%|                                                   | 0/100 [00:00<?, ?it/s]

368 Isolation and characterization of death domain (TNF-RI, Fas, TRADD, FADD/MORT-1, RIP) and TRAF domain-containing proteins (TRAF-1, TRAF-2, TRAF-3) have partially bridged a large molecular gap within one of several signaling pathways which originate at the plasma membrane and terminate in the nucleus. #
Prev size: 51 Vs. Next size 42
Prev size: 42 Vs. Next size 38
Prev size: 38 Vs. Next size 37
Prev size: 37 Vs. Next size 37


  1%|▍                                          | 1/100 [00:00<01:37,  1.01it/s]

528 The analysis of the viral proteins by electrophoresis indicates molecular weight differences between CDV and PDV in the fusion (F), phosphoprotein (P), H, nucleocapsid (N) and matrix (M) proteins. #
Prev size: 35 Vs. Next size 27
Prev size: 27 Vs. Next size 24
Prev size: 24 Vs. Next size 23
Prev size: 23 Vs. Next size 22


  2%|▊                                          | 2/100 [00:01<01:36,  1.02it/s]

...

Prev size: 17 Vs. Next size 17
163 However, when all of the four proteins (E-cadherin, alpha-catenin, beta-catenin, and gamma-catenin) were analysed as one group, a significant association was seen between reduction in immunoreactivity of at least one of these four proteins and the presence of metastases. Reduced or absent immunoreactivity in the tumour tissue was seen in 63 (70.0 per cent) for alpha-catenin, in 50 (55.6 per cent) for beta-catenin, and in 50 (55.6 per cent) for gamma-catenin. #
Prev size: 89 Vs. Next size 85


 99%|█████████████████████████████████████████▌| 99/100 [01:33<00:00,  1.05it/s]

Prev size: 85 Vs. Next size 85
493 RAD51, RAD52, RAD57, and RAD59 were required for efficient gap repair using either chromosomal or plasmid donors. #
Prev size: 23 Vs. Next size 21
Prev size: 21 Vs. Next size 20


100%|█████████████████████████████████████████| 100/100 [01:34<00:00,  1.06it/s]

Prev size: 20 Vs. Next size 20
CPU times: user 11min 15s, sys: 2min 30s, total: 13min 46s
Wall time: 1min 34s


In [16]:
from os import walk
directory = '/Volumes/GoogleDrive/My Drive/_My Data Analytics Exercise/Exercise/Biopython/CE/without/'
f = []
for (dirpath, dirnames, filenames) in walk(directory):
    f.extend(filenames)
    break
len(set(f)), f

(100,
 ['BioInfer_d390.gpickle',
  'BioInfer_d574.gpickle',
  'BioInfer_d834.gpickle',
  'BioInfer_d579.gpickle',
  'BioInfer_d434.gpickle',
  'BioInfer_d166.gpickle',
  'BioInfer_d201.gpickle',
  'BioInfer_d511.gpickle',
  'BioInfer_d616.gpickle',
  'IEPA_d101.gpickle',
  'BioInfer_d24.gpickle',
  'IEPA_d196.gpickle',
  'BioInfer_d200.gpickle',
  'IEPA_d92.gpickle',
  'BioInfer_d457.gpickle',
  'IEPA_d164.gpickle',
  'BioInfer_d438.gpickle',
  'IEPA_d26.gpickle',
  'IEPA_d88.gpickle',
  'BioInfer_d761.gpickle',
  'BioInfer_d293.gpickle',
  'BioInfer_d676.gpickle',
  'IEPA_d75.gpickle',
  'BioInfer_d697.gpickle',
  'BioInfer_d614.gpickle',
  'IEPA_d14.gpickle',
  'IEPA_d19.gpickle',
  'BioInfer_d679.gpickle',
  'IEPA_d29.gpickle',
  'BioInfer_d112.gpickle',
  'BioInfer_d11.gpickle',
  'BioInfer_d411.gpickle',
  'BioInfer_d699.gpickle',
  'BioInfer_d513.gpickle',
  'BioInfer_d316.gpickle',
  'BioInfer_d515.gpickle',
  'BioInfer_d483.gpickle',
  'BioInfer_d406.gpickle',
  'BioInfer_d677.